In [448]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import os

NOTE1: there are several duplicate records.  
NOTE2: one record (id: 537) has state CT but town is Vermont - possible that similar mistakes elsewhere.  
NOTE3: how to distinguish town/county level - may need a list for reference if want to step further.  
NOTE4: now need a manual check ('Haddam' =>'East Haddam')


## Cleaning town names

Goals:
- Clean town names that are different but refer to the same town in `ASD_all.xlsx` and `CD_all.xlsx`.
- check if all town names are in the "town-county" matching list given by `final_cw.xlsx` - if so, perform the matching.


Step 1: flag by 1 records that have NA "state" and "town" values.

In [449]:
# import and cleaning
ASD_all = pd.read_excel(
    'ASD_all.xlsx', index_col=0).reset_index(drop=True).dropna(how='all').drop_duplicates().reset_index(drop=True)
CD_all = pd.read_excel(
    'CD_all.xlsx', index_col=0).reset_index(drop=True).dropna(how='all').drop_duplicates().reset_index(drop=True)

# strip white space
for col in ['town', 'state', 'occupation']:
    ASD_all[col] = ASD_all[col].str.strip()
    CD_all[col] = CD_all[col].str.strip()

ASD_all['orig_town'] = ASD_all['town']
CD_all['orig_town'] = CD_all['town']
    
print(ASD_all.shape, CD_all.shape)

ASD_all

(5674, 10) (4337, 10)


,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents,town,state,occupation,orig_town
0,1064.00,75.0,532.00,37.0,508.00,51.0,Hartford,CT,Merchant,Hartford
1,449.00,96.0,224.00,97.0,232.00,10.0,Bolton,CT,Farmer,Bolton
2,154.00,20.0,77.00,10.0,192.00,NaN,Rhode Island,RI,Farmer,Rhode Island
3,196.00,75.0,98.00,37.0,172.00,24.0,Hartford,CT,Merchant,Hartford
4,53.00,58.0,26.00,79.0,67.00,6.0,Hartford,CT,Merchant,Hartford
...,...,...,...,...,...,...,...,...,...,...
5669,NaN,NaN,NaN,NaN,814.00,5.0,Chesterfield,NaN,NaN,Chesterfield
5670,NaN,NaN,NaN,NaN,28.00,29.0,Richmond,NaN,NaN,Richmond
5671,NaN,NaN,NaN,NaN,2030.00,35.0,New York,NaN,NaN,New York
5672,954.00,43.0,477.00,21.0,715.00,82.0,North Carolina,NaN,Administrator of McKennie Sumner,North Carolina


In [450]:
ASD_all['FLAG'], CD_all['FLAG'] = 0, 0

# identify NA rows
ASD_all.loc[(pd.isna(ASD_all['town'])) | (pd.isna(ASD_all['state'])), 'FLAG'] = 1
CD_all.loc[(pd.isna(CD_all['town'])) | (pd.isna(CD_all['state'])), 'FLAG'] = 1

# select non-NA subdataframe
ASD_no_NA = ASD_all[ASD_all['FLAG']==0]
CD_no_NA = CD_all[CD_all['FLAG']==0]

In [451]:
# check all states
print(
    set(list(ASD_no_NA.state.unique()) + list(CD_no_NA.state.unique()))
)

{'NJ', 'VA', 'DE', 'MD', 'NY', 'PA', 'GB', 'VT', 'US', 'GA', 'VI', 'BVI', 'FR', 'FC', 'SC', 'MA', 'BM', 'RI', 'NC', 'NH', 'CT'}


Step 2: flag by 2 the records with states outside the following list of states.

In [452]:
state_list = ['RI', 'CT', 'GA', 'MD', 'NC', 'NH', 'NJ', 'NY', 'VA', 'PA', 'RI', 'SC', 'DE', 'MA', 'VT']
state_name_list = [
            'Rhode Island', 'Connecticut', 'Georgia', 'Maryland', 'North Carolina', 
            'New Hampshire', 'New Jersey', 'New York', 'Virginia',
            'Pennsylvania', 'Rhode Island', 'South Carolina', 'Delaware', 
            'Massachusetts', 'Vermont'
            ]
state_list_dict = dict(zip(state_list, state_name_list))

ASD_no_NA.loc[ASD_no_NA.apply(lambda row: row.state not in state_list, axis=1), 'FLAG'] = 2
CD_no_NA.loc[CD_no_NA.apply(lambda row: row.state not in state_list, axis=1), 'FLAG'] = 2

# update to the original df
ASD_all.update(ASD_no_NA)
CD_all.update(CD_no_NA)


ASD_all[ASD_all['FLAG']==2].head()

,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents,town,state,occupation,orig_town,FLAG
1748,19.0,NaN,9.0,4.000000,NaN,NaN,Eustatia,BVI,NaN,Eustatia,2.0
1749,125.0,45.000000,62.0,73.000000,134.0,87.00,Eustatia,BVI,NaN,Eustatia,2.0
1750,90.0,NaN,45.0,NaN,93.0,84.00,Eustatia,BVI,NaN,Eustatia,2.0
4265,44482.0,-0.106667,44482.0,-0.053333,44482.0,0.42,London,GB,Merchant,London,2.0
4325,2346.0,-0.400000,2346.0,-0.200000,2346.0,0.20,London,GB,Merchant,London,2.0


Step 3: flag by 3 the records
- with town name 'State of XX' or 'State XX' or 'XX State' where XX is the state where the town is located, and 
- with town name exactly or almost the same as the state name (due to typos).

These are records for which "township" is not available.

In [453]:
ASD_rest = ASD_all[ASD_all['FLAG']==0]
CD_rest = CD_all[CD_all['FLAG']==0]

# state of XX/state XX - checked
ASD_rest.loc[
    ASD_rest.apply(lambda row: row.town.lower().startswith('state '), axis=1), 'FLAG'] = 3
CD_rest.loc[
    CD_rest.apply(lambda row: row.town.lower().startswith('state '), axis=1), 'FLAG'] = 3
ASD_rest.loc[ASD_rest.town=='Delaware State', 'FLAG'] = 3
CD_rest.loc[CD_rest.town=='Delaware State', 'FLAG'] = 3

# town name == state name - checked
ASD_rest.loc[ASD_rest.apply(lambda row: row.town==state_list_dict[row.state], axis=1), 'FLAG'] = 3
CD_rest.loc[CD_rest.apply(lambda row: row.town==state_list_dict[row.state], axis=1), 'FLAG'] = 3

# town name ~= state name - checked
ASD_rest.loc[
    ASD_rest.apply(lambda row: 
    process.cdist([row.town], [state_list_dict[row.state]])[0][0] >= 80, axis=1), 'FLAG'
    ] = 3
CD_rest.loc[
    CD_rest.apply(lambda row: 
    process.cdist([row.town], [state_list_dict[row.state]])[0][0] >= 80, axis=1), 'FLAG'
    ] = 3

# Carolina in South Carolina (no county named Carolina)
ASD_rest.loc[
    ASD_rest.apply(lambda row: row.town in state_list_dict[row.state] and
    row.town != state_list_dict[row.state], axis=1), 'FLAG'] = 3
CD_rest.loc[
    CD_rest.apply(lambda row: row.town in state_list_dict[row.state] and
    row.town != state_list_dict[row.state], axis=1), 'FLAG'] = 3

# update
ASD_all.update(ASD_rest)
CD_all.update(CD_rest)
ASD_all[ASD_all['FLAG']==3].head()

,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents,town,state,occupation,orig_town,FLAG
2,154.0,20.0,77.0,10.0,192.0,NaN,Rhode Island,RI,Farmer,Rhode Island,3.0
35,60.0,37.0,30.0,18.0,63.0,83.0,Rhode Island,RI,NaN,Rhode Island,3.0
102,248.0,46.0,124.0,23.0,144.0,NaN,State of New York,NY,NaN,State of New York,3.0
122,738.0,43.0,369.0,22.0,708.0,13.0,State of New York,NY,NaN,State of New York,3.0
167,30.0,48.0,15.0,24.0,8.0,22.0,State of Vermont,VT,NaN,State of Vermont,3.0


Step 4: unify town names
- remove `"state of" + state_name` and `"of" + state_name` that appear in the town name, except "City of New York",
- remove `"state" + state_name` that appears in the town name,
- remove `state_name` from `XX + state_name` or `state_name + XX`,
- take into account the three special cases.

Then, 
- create a new identifier and remove "Town"/"County" from town names.

Flag this change by 4.

In [454]:
ASD_rest = ASD_all[ASD_all.FLAG==0]
CD_rest = CD_all[CD_all.FLAG==0]

def remove_state_from_town(row):
    state_name = state_list_dict[row.state]
    # state of/of
    row.town = row.town.replace(' State of ' + state_name, '')
    # state + state_name
    row.town = row.town.replace(' State ' + state_name, '')
    # special cases
    if row.town == 'Boston state Massachusetts':
        row.town = 'Boston'

    if row.town == 'New Castle County Delaware State':
        row.town = 'New Castle County'

    if row.town == 'Virginia and Philadelphia':
        row.town = 'Philadelphia'

    if row.town != 'City of New York':
        row.town = row.town.replace(' of ' + state_name, '')
        row.town = row.town.replace(state_name, '')
    
    # flag changes
    row.FLAG = 4
    return row

ASD_rest = ASD_rest.apply(lambda row: remove_state_from_town(row), axis=1)
CD_rest = CD_rest.apply(lambda row: remove_state_from_town(row), axis=1)

# town_level = 'T' if specified 'Town', 'C' if specified 'County', otherwise 'U'
ASD_rest['town_level'] = 'U'
ASD_rest.loc[ASD_rest['town'].str.contains('County', na=False), 'town_level'] = 'C'
ASD_rest.loc[ASD_rest['town'].str.contains('Town', na=False), 'town_level'] = 'T'

def remove_CountyTown(row):
    row.town = row.town.replace('County', '')
    row.town = row.town.replace('Town', '')
    row.FLAG = 4
    return row
    
ASD_rest = ASD_rest.apply(lambda row: remove_CountyTown(row), axis=1)
CD_rest = CD_rest.apply(lambda row: remove_CountyTown(row), axis=1)


ASD_all.update(ASD_rest)
CD_all.update(CD_rest)

Step 5: match town names that are likely to be the same one  
1. for each state, create a list `A` of town names with # of occurences >= 4; create a list `B` for all the rest towns.
2. for each town in list `B`, find the best three matches with towns in list `A` - if above a threshold, report all three and match to the best one. - to be checked afterwards
3. all unmatched towns in list `B` become a new list `C` - we WANT to compare one another and if the similarity is above a threshold, group them; otherwise keep it untouched. One simple procedure is for each town in list `C`, group it with all other towns whose distance to it is smaller than a threshold. Then proceed to the next one if it's not in some group already and skip otherwise. This is legitimate because we expect "typos" to cause small differences among all mistyped names. Report all the matched and unmatched cases in this round.
4. For all above, allow user's input to manually confirm the matches.

Flag this change by 5.

In [455]:
def match_towns(df):
    if len(df.index)==0:    # Vermont
        return df
    print(f"State: {df.state.iloc[0]}")

    val_counts = df.town.value_counts(sort=True)
    list_A, list_B = list(val_counts[val_counts>=min(val_counts.iloc[0], 4)].index), list(val_counts[val_counts<min(val_counts.iloc[0], 6)].index)
    list_C = list_B.copy()
    for town in list_B:
        best3 = process.extract(town, list_A)[0:3]
        if best3[0][1] >= 85:
            print(f"{town}. Candidates: {[x[0] for x in best3]}. Matched: {best3[0][0]}.\n")

            # # user input
            # ACCEPT = input("1 for ACCEPT. 0 for REJECT")
            # # records

            # if ACCEPT==1:
            # matched to list_A
            list_C.remove(town)
            df.loc[df.town==town, 'FLAG'] = 5
            df.loc[df.town==town, 'town'] = best3[0][0]

    list_C_flag = [-1 for x in list_C]
    for id, town in enumerate(list_C):
        # only do matching if not already matched
        if list_C_flag[id] == -1:
            bests = [x[0] for x in process.extract(town, list_C, score_cutoff=85)]
            if len(bests) > 1:
                # if not just oneself
                indexes = [list_C.index(x) for x in bests]
                # make sure same group has the same id
                print(f"Candidate group: {bests}")

                # pick the shortest one as the name we want to keep
                # this deals with the cases like North Hampton -> Hampton
                index_selected = list_C.index(min(bests, key=len))
                
                for k in indexes: 
                    list_C_flag[k] = index_selected 

                # one special case
                if bests==['Cumberland', 'Cumb  ']:
                    list_C_flag[k] = list_C.index('Cumberland')
    
    for id, flag in enumerate(list_C_flag):
        if flag != -1:
            df.loc[df.town==list_C[id], 'FLAG'] = 5
            df.loc[df.town==list_C[id], 'town'] = list_C[flag]

    return df

In [456]:
for state_code in state_list:
    
    ASD_rest[ASD_rest.state==state_code] = match_towns(ASD_rest[ASD_rest.state==state_code])
    CD_rest[CD_rest.state==state_code] = match_towns(CD_rest[CD_rest.state==state_code])


# deal with some other special cases
ASD_rest.loc[ASD_rest.town=='East Haddam', 'town'] = 'Haddam'
CD_rest.loc[CD_rest.town=='East Haddam', 'town'] = 'Haddam'

ASD_all.update(ASD_rest)
CD_all.update(CD_rest)

State: RI
Johnson. Candidates: ['Johnson', 'Johnston', 'South Kingston']. Matched: Johnson.

Portsmouth. Candidates: ['Portsmouth', 'North Providence', 'South Kingston']. Matched: Portsmouth.

Richmond. Candidates: ['Richmond', 'East Greenwich', 'West Greenwich']. Matched: Richmond.

Conventry. Candidates: ['Coventry', 'Exeter', 'North Providence']. Matched: Coventry.

Glocester. Candidates: ['Gloucester', 'Exeter', 'Coventry']. Matched: Gloucester.

North Kingstone. Candidates: ['North Kingston', 'South Kingston', 'Newport']. Matched: North Kingston.

North Kingstown. Candidates: ['North Kingston', 'South Kingston', 'Newport']. Matched: North Kingston.

Stoughton now of Providence. Candidates: ['Providence', 'North Providence', 'Johnston']. Matched: Providence.

Smithfeild. Candidates: ['Smithfield', 'Scituate', 'Cumberland']. Matched: Smithfield.

South Kingstone. Candidates: ['South Kingston', 'North Kingston', 'Johnston']. Matched: South Kingston.

Warnick. Candidates: ['Warwick', 

### Now maps to the County-town list

There are a few cases where the `orig_town` names include `'County'`, but gets mapped to nothing on the list. We assign `county` to `name_type` in this case, and let `county` be county name.

In [457]:
ASD_no_NA = ASD_all[ASD_all['FLAG']!=1]
CD_no_NA = CD_all[CD_all['FLAG']!=1]

matchlist = pd.read_csv('countymapping.csv', index_col=0)
matchlist = matchlist.rename({'town': 'orig_town'}, axis=1)
matchlist.head()

,orig_town,state,county,name_type
0,Hartford,CT,Hartford County,town
1,Bolton,CT,Tolland County,town
2,Rhode Island,RI,NaN,state
5,Wethersfield,CT,Hartford County,town
6,Lyme,CT,New London County,town


In [458]:
ASD_merged = pd.merge(left=ASD_no_NA, right=matchlist, how='left', on=['state', 'orig_town'])
CD_merged = pd.merge(left=ASD_no_NA, right=matchlist, how='left', on=['state', 'orig_town'])


# select
filter_ASD = ASD_merged.orig_town.str.contains('County') & ASD_merged.name_type.isna()
filter_CD = CD_merged.orig_town.str.contains('County') & CD_merged.name_type.isna()
ASD_merged.loc[filter_ASD]

,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents,town,state,occupation,orig_town,FLAG,county,name_type
2155,80.0,28.00,40.0,15.00,28.0,90.00,Berks,PA,NaN,Pennsylvania Berks County,5.0,NaN,NaN
2156,511.0,63.00,255.0,81.00,230.0,13.00,Berks,PA,NaN,Pennsylvania Berks County,5.0,NaN,NaN
2157,233.0,10.00,116.0,55.00,112.0,8.00,Montgomery,PA,NaN,Pennsylvania Montgomery County,5.0,NaN,NaN
2680,253.0,49.00,126.0,74.00,290.0,23.00,Northumberland,PA,NaN,Northumberland County Virginia,5.0,NaN,NaN
4037,1634.0,-0.14,1634.0,-0.08,1634.0,0.38,Pendleton,SC,Planter,Pendleton County,4.0,NaN,NaN


In [459]:
ASD_merged.loc[filter, 'name_type'] = 'county'
ASD_merged.loc[filter, 'county'] = ASD_merged.loc[filter, 'town']

CD_merged.loc[filter, 'name_type'] = 'county'
CD_merged.loc[filter, 'county'] = CD_merged.loc[filter, 'town']

# remove "County" from county name
ASD_merged['county'] = ASD_merged['county'].apply(lambda x: x.replace('County', '').rstrip() if type(x)==str else x)
CD_merged['county'] = CD_merged['county'].apply(lambda x: x.replace('County', '').rstrip() if type(x)==str else x)

ASD_merged

,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents,town,state,occupation,orig_town,FLAG,county,name_type
0,1064.0,75.00,532.0,37.00,508.0,51.00,Hartford,CT,Merchant,Hartford,4.0,Hartford,town
1,449.0,96.00,224.0,97.00,232.0,10.00,Bolton,CT,Farmer,Bolton,4.0,Tolland,town
2,154.0,20.00,77.0,10.00,192.0,NaN,Rhode Island,RI,Farmer,Rhode Island,3.0,NaN,state
3,196.0,75.00,98.0,37.00,172.0,24.00,Hartford,CT,Merchant,Hartford,4.0,Hartford,town
4,53.0,58.00,26.0,79.00,67.0,6.00,Hartford,CT,Merchant,Hartford,4.0,Hartford,town
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4069,112.0,-0.40,112.0,NaN,112.0,NaN,Charleston,SC,NaN,Charleston,4.0,Charleston,town
4070,978.0,-0.20,978.0,-0.47,978.0,-0.21,North Carolina,NC,NaN,North Carolina,3.0,NaN,state
4071,1900.0,0.00,1900.0,-0.24,1900.0,0.00,Charleston,SC,Merchant,Charleston,4.0,Charleston,town
4072,1106.0,0.00,1106.0,NaN,1106.0,NaN,Charleston,SC,Merchant,Charleston,4.0,Charleston,town


In [460]:
# assign nan to town if name_type == county/state
ASD_merged.loc[(ASD_merged.name_type=='county') | (ASD_merged.name_type=='state'), 'town'] = np.nan
CD_merged.loc[(CD_merged.name_type=='county') | (CD_merged.name_type=='state'), 'town'] = np.nan

ASD_df= ASD_merged.drop(['orig_town'], axis=1)
CD_df = CD_merged.drop(['orig_town'], axis=1)

# remove na cases and others
ASD_df = ASD_df.loc[(~ASD_merged.name_type.isna()) & (ASD_merged.name_type!='other')]
CD_df = CD_df.loc[(~CD_merged.name_type.isna()) & (CD_merged.name_type!='other')]

ASD_df = ASD_df.fillna('NaN')
CD_df = CD_df.fillna('NaN')

In [461]:
ASD_df.groupby(['state', 'county', 'town'], dropna=False).size().to_csv('ASD_counts.csv')
CD_df.groupby(['state', 'county', 'town'], dropna=False).size().to_csv('CD_counts.csv')